In [2]:
#4) Here is done the training of the 4th exercise
import tensorflow as tf
import numpy as np
%matplotlib inline  
import matplotlib.pyplot as plt

In [21]:
Xdata = np.load("../data/hw2-train-data.npy")
Ydata = np.load("../data/hw2-train-labels.npy").reshape(-1,1)
print("data shape = ", Xdata.shape)
print("labels shape = ", Ydata.shape)

n = 50000
Xtrn,Ytrn = (Xdata[:n]-np.mean(Xdata[:n]))/np.std(Xdata[:n]),np.squeeze(Ydata[:n]) # half of the points for training
print(np.squeeze(Ytrn))
ntrn,dim = Xtrn.shape
Xval,Yval = (Xdata[n:]-np.mean(Xdata[n:]))/np.std(Xdata[n:]),np.squeeze(Ydata[n:]) # rest for model evaluation


('data shape = ', (57500, 768))
('labels shape = ', (57500, 1))
[5 0 4 ..., 8 4 8]


In [73]:
momentum = 0.9
learning_rate_1 = 0.001
steps = 10
batch = 25
epochs = 10
inp_dims = 768

In [74]:
def cnn_1d(X, y, k, l, learning_rate):
    
    X = tf.expand_dims(X,-1)
    
    for i in range(k):
        X = tf.layers.conv1d(X, 32 , 5 , activation = tf.nn.relu)
        X = tf.layers.max_pooling1d(X, 3, 2)
    
    X = tf.nn.dropout(X, keep_prob=0.7)
    X = tf.layers.flatten(X)
    
    for i in range(l):
        X = tf.layers.dense(X, 100, activation = tf.nn.relu)
    
    logits = tf.layers.dense(X, 10, activation = None)
    softmax_ = tf.nn.softmax(logits)
    
    xentropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y,10, dtype=tf.float32), logits=logits))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(xentropy)
    correct_prediction = tf.equal(tf.argmax(softmax_, 1), tf.cast(y, tf.int64))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return xentropy, accuracy, train_step

In [75]:
def stoch(k ,l ,batch = batch, epochs = epochs, learning_rate = learning_rate_1, momentum = 0.0, only_last = False, optimizer = "None"):
    print("k =",k, "l =",l,"batch =",batch,"learning_rate =",learning_rate,"momentum = ", momentum)
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, shape=(None, inp_dims), name="X")
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    loss, acc, up = cnn_1d(X, y, k, l, learning_rate)
    perms = np.array_split(np.random.permutation(len(Xtrn)),len(Xtrn)/batch)
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        for i in range(epochs):
            for perm in perms:
                if optimizer not in ["adam","gradient"]:
                    loss_train, acc_train, _, _, _ = sess.run([loss,acc,up[0],up[1],up[2]], feed_dict={X: Xtrn[perm], y:Ytrn[perm]})
                else:
                    loss_train, acc_train, _ = sess.run([loss,acc,up], feed_dict={X: Xtrn[perm], y:Ytrn[perm]})
            loss_val, acc_val = sess.run([loss, acc], feed_dict={X: Xval, y:Yval})
            if only_last == False:
                print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "accuracy(train)", acc_train, "accuracy(val)", acc_val)
        if only_last == True:
            print("step", i, "loss(train)", loss_train, "loss(val)", loss_val, "accuracy(train)", acc_train, "accuracy(val)", acc_val)


In [76]:
[stoch( k = k_, l = l_, optimizer = "adam") for k_ in range(6) for l_ in range(4)] 

('k =', 0, 'l =', 0, 'batch =', 25, 'learning_rate =', 0.001, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.20987314, 'loss(val)', 0.35601777, 'accuracy(train)', 0.88, 'accuracy(val)', 0.91013336)
('step', 1, 'loss(train)', 0.16765392, 'loss(val)', 0.35238093, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91213334)
('step', 2, 'loss(train)', 0.15175505, 'loss(val)', 0.35312095, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91333336)
('step', 3, 'loss(train)', 0.14031637, 'loss(val)', 0.3596496, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91293335)
('step', 4, 'loss(train)', 0.13427378, 'loss(val)', 0.37089375, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91240001)
('step', 5, 'loss(train)', 0.12910408, 'loss(val)', 0.3779164, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91106665)
('step', 6, 'loss(train)', 0.12517706, 'loss(val)', 0.38901636, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.91053331)
('step', 7, 'loss(train)', 0.12636536, 'loss(val)', 0.4029

('step', 4, 'loss(train)', 0.00070125202, 'loss(val)', 0.13211364, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96799999)
('step', 5, 'loss(train)', 0.00035251625, 'loss(val)', 0.12007433, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96906668)
('step', 6, 'loss(train)', 7.6196186e-05, 'loss(val)', 0.15142265, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96706665)
('step', 7, 'loss(train)', 2.0211746e-05, 'loss(val)', 0.12309309, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97280002)
('step', 8, 'loss(train)', 7.5624148e-06, 'loss(val)', 0.14415632, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97079998)
('step', 9, 'loss(train)', 0.00011458521, 'loss(val)', 0.18863779, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96906668)
('k =', 1, 'l =', 3, 'batch =', 25, 'learning_rate =', 0.001, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.069238842, 'loss(val)', 0.15530813, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.95279998)
('step', 1, 'loss(train)', 0.046797927, 'loss(val)', 0.13341846, 'accurac

('step', 9, 'loss(train)', 0.018658616, 'loss(val)', 0.19315189, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96693331)
('k =', 3, 'l =', 1, 'batch =', 25, 'learning_rate =', 0.001, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.012053343, 'loss(val)', 0.11816691, 'accuracy(train)', 1.0, 'accuracy(val)', 0.96373332)
('step', 1, 'loss(train)', 0.015194745, 'loss(val)', 0.10064564, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97000003)
('step', 2, 'loss(train)', 0.01706033, 'loss(val)', 0.099706344, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97240001)
('step', 3, 'loss(train)', 0.00020554628, 'loss(val)', 0.099465206, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97240001)
('step', 4, 'loss(train)', 0.0017459792, 'loss(val)', 0.1139145, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97146666)
('step', 5, 'loss(train)', 0.00043595012, 'loss(val)', 0.10015022, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97680002)
('step', 6, 'loss(train)', 0.00027078952, 'loss(val)', 0.11193456, 'accuracy(train)', 1

('step', 3, 'loss(train)', 0.024360538, 'loss(val)', 0.10408535, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97320002)
('step', 4, 'loss(train)', 0.010762543, 'loss(val)', 0.1112451, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97359997)
('step', 5, 'loss(train)', 0.27444351, 'loss(val)', 0.10028583, 'accuracy(train)', 0.92000002, 'accuracy(val)', 0.97546667)
('step', 6, 'loss(train)', 0.088712595, 'loss(val)', 0.091769524, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.97653335)
('step', 7, 'loss(train)', 0.018012449, 'loss(val)', 0.093295664, 'accuracy(train)', 1.0, 'accuracy(val)', 0.97973335)
('step', 8, 'loss(train)', 0.28109074, 'loss(val)', 0.10497115, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.97586668)
('step', 9, 'loss(train)', 0.38521889, 'loss(val)', 0.10154617, 'accuracy(train)', 0.95999998, 'accuracy(val)', 0.97826666)
('k =', 5, 'l =', 0, 'batch =', 25, 'learning_rate =', 0.001, 'momentum = ', 0.0)
('step', 0, 'loss(train)', 0.07906235, 'loss(val)', 0.1200096, 'ac

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]